In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
X = pd.read_pickle('../../data/preprocessed/merged/mergedX.pkl')

In [3]:
# Apply label encoding to X
label_encoders = {}
string_columns = X.select_dtypes(include=['object'])
for column in string_columns:
    label_encoders[column] = preprocessing.LabelEncoder().fit(string_columns[column])
    X[column] = label_encoders[column].transform(string_columns[column])

In [4]:
# Load Y and merge with X
Y = pd.read_csv('../../data/preprocessed/merged/Y.csv')
Y = Y.merge(X, how='right', on='SEQN').drop(columns=[column for column in list(X.columns) if column != 'SEQN'])

In [5]:
Y = Y.drop(columns=['Unnamed: 0'])
Y = Y.applymap(lambda x: int(x))

In [6]:
y_cleaned = Y[Y.columns[1:]].as_matrix()
y_cleaned

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [7]:
# Fit Random Forest with X and Y (removed SEQN)
clf = RandomForestClassifier(n_estimators=1000, max_depth=5,random_state=0)
clf.fit(X[X.columns[1:]].as_matrix(), Y[Y.columns[1:]].as_matrix())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [8]:
scores = cross_val_score(clf, X[X.columns[1:]].as_matrix(), Y[Y.columns[1:]].as_matrix(), cv=10, scoring='f1_weighted')

In [9]:
scores

array([0.03583062, 0.0130719 , 0.02614379, 0.03267974, 0.03921569,
       0.03267974, 0.03267974, 0.10457516, 0.03594771, 0.09150327])